In [ ]:
# import library
library(dplyr)

In [ ]:
# read datasets
matches = read.csv("matches.csv")
deli = read.csv("deliveries.csv")

# Question 1:

In [ ]:
# teams playing today's match
teams = c("Sunrisers Hyderabad", "Deccan Chargers", "Delhi Capitals", "Delhi Daredevils")

In [ ]:
# all matches between SRH and DC
srh_dc = subset(matches, (team1 %in% teams & team2 %in% teams))

In [ ]:
srh_dc$winner

In [ ]:
# victories of the teams against each other
count(srh_dc, winner)

In [ ]:
# matches of SRH
srh = subset(matches, (team1 %in% c("Sunrisers Hyderabad", "Deccan Chargers") | team2 %in% c("Sunrisers Hyderabad", "Deccan Chargers") ))

In [ ]:
# total no. of matches played by SRH
nrow(srh)

In [ ]:
# winners in those matches
count(srh, winner)

In [ ]:
# win ratio of SRH
(29+58)/183

In [ ]:
# matches of DC
dc = subset(matches, (team1 %in% c("Delhi Daredevils", "Delhi Capitals") | team2 %in% c("Delhi Daredevils", "Delhi Capitals") ))

In [ ]:
# total no. of matches played by DC
nrow(dc)

In [ ]:
# winners in those matches
count(dc, winner)

In [ ]:
# win ratio of DC
(67+10)/177

# Question 2:

In [ ]:
# all deliveries faced by SRH
srh_bat = subset(deli, batting_team %in% c('Sunrisers Hyderabad', 'Deccan Chargers'))

head(srh_bat)

In [ ]:
# all deliveries faced by DC
dc_bat = subset(deli, batting_team %in% c("Delhi Daredevils", "Delhi Capitals"))

head(dc_bat)

In [ ]:
# function to calculate balls taken to score the given runs
get_balls_for_runs = function(df, runs){
    
    # empty list to store the number of balls
    num_balls = numeric()
    
    # iterating over all the matches
    for (m_id in unique(df$match_id)){
        
        # getting the record of one match using the match id
        data = subset(df, (match_id == m_id))
        
        # calculating cumulative sum of the score
        cumsum = cumsum(data$total_runs)
        
        # finding the index position of the ball on which the score of runs is crossed
        end_pos = which.max(data.frame(cumsum >= runs)$cumsum....runs)
        
        # dataframe which stores the details of the balls below the score of 100
        balls_taken_df = data[1:end_pos, ]
        
        # total balls taken
        total_balls = nrow(balls_taken_df)
        # extra balls like wide, no ball, etc. 
        extra_balls = nrow(subset(balls_taken_df, (extra_runs != 0)))
        
        # final number of balls
        final_balls = total_balls - extra_balls
        num_balls = c(num_balls, final_balls)
        
        # removing the cases where the score of given runs was not crossed
        final = num_balls[num_balls>1]
        
        
    }
    # returning the result
    return (final)
}

In [ ]:
# balls taken to score a 30 considering all the matches of DC
num_30 = get_balls_for_runs(dc_bat, 30)

summary(num_30)

# balls taken to score a 50 considering all the matches of DC
num_50 = get_balls_for_runs(dc_bat, 50)

summary(num_50)

In [ ]:
# balls taken to score a 30 considering all the matches of SRH
num_30 = get_balls_for_runs(srh_bat, 30)

summary(num_30)

# balls taken to score a 50 considering all the matches of SRH
num_50 = get_balls_for_runs(srh_bat, 50)

summary(num_50)

In [ ]:
# all deliveries in SRH vs DC matches
srh_dc_deli = subset(deli, (bowling_team %in% teams & batting_team %in% teams))

head(srh_dc_deli)

In [ ]:
# all deliveries faced by DC against SRH
dc_bat_srh = subset(srh_dc_deli, batting_team %in% c("Delhi Daredevils", "Delhi Capitals") )

# balls taken to score a 30 considering all the matches of DC against SRH
num_30 = get_balls_for_runs(dc_bat_srh, 30)

summary(num_30)

# balls taken to score a 50 considering all the matches of DC against SRH
num_50 = get_balls_for_runs(dc_bat_srh, 50)

summary(num_50)

In [ ]:
# all deliveries faced by SRH against DC
srh_bat_dc = subset(srh_dc_deli, batting_team %in% c('Sunrisers Hyderabad', 'Deccan Chargers'))

# balls taken to score a 30 considering all the matches of SRH against DC
num_30 = get_balls_for_runs(srh_bat_dc, 30)

summary(num_30)

# balls taken to score a 50 considering all the matches of SRH against DC
num_50 = get_balls_for_runs(srh_bat_dc, 50)

summary(num_50)

# Question 3:

In [ ]:
# function to get the runs scored on the last 7 balls played by Warner
get_runs = function(df){
    
    # filtering out the deliveries for Warner
    data = subset(df, with(df, grepl('DA Warner', batsman)))
    
    # ids of all the matches that he has played
    matches = unique(data$match_id)
    
    # variable to store the number of matches in which he did not play 7 balls
    count = 0
    # list to store the runs
    runs_per_match = numeric()
    
    # iterating over the matches
    for (m_id in matches){
        # all deliveries played by Warner in that specific match
        match_data = subset(data, match_id == m_id)
        # if he plays less than 7 balls, increase the value of the variable "count"
        if (nrow(match_data) < 7){
            count = count + 1
            # otherwise, calculate the sum of runs on the last 7 balls he played
        } else {
            runs = sum(tail(match_data, 7)$batsman_runs)    # filtering the last 7 balls and calculating the sum
            runs_per_match = c(runs_per_match, runs)        # adding to the plist
        }
    }
    # printing the results
    print("Total number of matches")
    print(length(matches))
    print("Number of matches in which he did not play 7 or more balls")
    print(count)
    # returning the results
    return (runs_per_match)
}

In [ ]:
# getting the runs scored by Warner in IPL history
runs = get_runs(deli)

# histogram
hist(runs, freq=TRUE, breaks=c(0, 8, 13, 17, max(runs)))

In [ ]:
# getting the runs scored by Warner in IPL 2019
runs_19 = get_runs(subset(deli, match_id > 11000))

# histogram
hist(runs_19, freq=TRUE, breaks=c(0, 8, 13, 17, max(runs_19)))

In [ ]:
# getting the runs scored by Warner against DC
runs_dc = get_runs(subset(deli, batting_team %in% c("Delhi Daredevils", "Delhi Capitals")))

# histogram
hist(runs_dc, freq=TRUE, breaks=c(0, 8, 13, 17, max(runs_dc)))

# Question 4:

In [ ]:
# batsmen playing today's match
batsmen = c('DA Warner', 'WP Saha', 'MK Pandey', 'KS Williamson', 'JO Holder',
           'S Dhawan', 'AM Rahane', 'SS Iyer', 'RR Pant', 'S Hetmyer', 'P Shaw', 'MP Stoinis')

In [ ]:
# function to get the boundaries
get_boundaries = function(bname, df){
    # filtering out the deliveries for each batsman
    data = subset(df, with(df, grepl(bname, batsman)))
    # 4s hit
    runs_4 = subset(data, batsman_runs == 4) %>% group_by(match_id) %>% summarise(runs_4 = n())
    # 6s hit
    runs_6 = subset(data, batsman_runs == 6) %>% group_by(match_id) %>% summarise(runs_6 = n())
    # merging the dataframes
    boundaries = merge(runs_4, runs_6, all=TRUE)
    # filling the null values with 0
    boundaries[is.na(boundaries)] = 0
    # finding out the total number of boundaries
    boundaries$boundaries = boundaries$runs_4 + boundaries$runs_6
    
    # returning the results
    return(summary(boundaries$boundaries))
}

## Performance of the batsmen in IPL history

In [ ]:
# for loop to iterate over the batsman names
for (b_name in batsmen){
    # print batsman name
    print(b_name)
    # printing the statistical measures 
    print(get_boundaries(b_name, deli))
}

## Performance of the batsmen in IPL 2019

In [ ]:
# for loop to iterate over the batsman names
for (b_name in batsmen){
    # print batsman name
    print(b_name)
    # printing the statistical measures 
    print(get_boundaries(b_name, subset(deli, match_id > 11000)))
}

# Question 5:

In [ ]:
# all deliveries bowled by Rashid Khan
rashid = subset(deli, with(deli, grepl("Rashid Khan", bowler)) )

head(rashid)

In [ ]:
# wickets taken by Rashid in each match
wickets = subset(rashid, player_dismissed != '') %>% group_by(match_id) %>% summarise(num_wickets = n())

head(wickets)

In [ ]:
# total matches Rashid has played in IPL
length(unique(rashid$match_id))

# number of matches in which he could not take a wicket
length(unique(rashid$match_id)) - nrow(wickets)

In [ ]:
# frequency table of wickets
table(wickets$num_wickets)

In [ ]:
# wickets taken by Rashid in each match in IPL 2019
wickets_2019 = subset(rashid, match_id>11000 & player_dismissed != '') %>% group_by(match_id) %>% summarise(num_wickets = n())

In [ ]:
# total matches Rashid has played in IPL 2019
length(unique(subset(rashid, match_id>11000)$match_id))

# number of matches in which he could not take a wicket
length(unique(subset(rashid, match_id>11000)$match_id)) - nrow(wickets_2019)

In [ ]:
# frequency table of wickets
table(wickets_2019$num_wickets)

In [ ]:
# wickets taken by Rashid in each match against DC
wickets_dc = subset(rashid, batting_team %in% c("Delhi Daredevils", "Delhi Capitals") & player_dismissed != '') %>% group_by(match_id) %>% summarise(num_wickets = n())

In [ ]:
# total matches Rashid has played against DC
length(unique(subset(rashid, batting_team %in% c("Delhi Daredevils", "Delhi Capitals"))$match_id))

# number of matches in which he could not take a wicket
length(unique(subset(rashid, batting_team %in% c("Delhi Daredevils", "Delhi Capitals"))$match_id)) - nrow(wickets_dc)

In [ ]:
# frequency table of wickets
table(wickets_dc$num_wickets)